In [ ]:
from submit import my_fit, my_predict
import time as tm
import pickle
import warnings
import os

In [ ]:
with open( "dict_secret", 'r' ) as f:
	words = f.read().split( '\n' )[:-1]		# Omit the last line since it is empty
	num_words = len( words )

In [ ]:
n_trials = 5

t_train = 0
m_size = 0
t_test = 0
prec = 0

In [ ]:
def get_bigrams( word, lim = None ):
  # Get all bigrams
  bg = map( ''.join, list( zip( word, word[1:] ) ) )
  # Remove duplicates and sort them
  bg = sorted( set( bg ) )
  # Make them into an immutable tuple and retain only the first few
  return tuple( bg )[:lim]

In [ ]:
lim_bg = 5
lim_out = 5

for t in range( n_trials ):
  tic = tm.perf_counter()
  model = my_fit( words )
  toc = tm.perf_counter()
  t_train += toc - tic

  with open( f"model_dump_{t}.pkl", "wb" ) as outfile:
    pickle.dump( model, outfile, protocol = pickle.HIGHEST_PROTOCOL )

  m_size += os.path.getsize( f"model_dump_{t}.pkl" )

  tic = tm.perf_counter()

  for ( i, word ) in enumerate( words ):
    bg = get_bigrams( word, lim = lim_bg )
    guess_list = my_predict( model, bg )

    # Do not send long guess lists -- they will result in lower marks
    guess_len = len( guess_list )
    # Ignore all but the first 5 guesses
    guess_list = guess_list[ :lim_out ]

    # Notice that if 10 guesses are made, one of which is correct,
    # score goes up by 1/10 even though only first 5 guesses are considered
    # Thus, it is never beneficial to send more than 5 guesses
    if word in guess_list:
      prec += 1 / guess_len

  toc = tm.perf_counter()

  t_test += toc - tic

In [ ]:
t_train /= n_trials
m_size /= n_trials
t_test /= n_trials
prec /= ( n_trials * num_words )

print( t_train, m_size, prec, t_test )